# SPARQL queries & tests

## Database password
Please fill in the database password below

In [1]:
from __future__ import print_function
import ipywidgets as widgets
passwordField = widgets.Password(value="postgres")
passwordField

Password()

In [2]:
from sqlalchemy import create_engine

connectionString = "postgresql://postgres:%s@postgresdb:5432/thunder" % passwordField.value
dbEngine = create_engine(connectionString)

/usr/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from r2rmlQueryEditor import R2RMLQueryEditor

r2rmlEditor = R2RMLQueryEditor("http://sparql_open:7200/repositories/r2rml")
widgets.VBox(r2rmlEditor.runInterface())

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oVGV4dGFyZWEodmFsdWU9dSdTRUxFQ1QgaWQsIGdlbmRlciBGUk9NIHRodW5kZXJzZXQnLCBwbGFjZWhvbGRlcj11J2h0dHA6Ly9tYXBwaW7igKY=


In [4]:
from comparisonEngine import ComparisonEngine

comparisonEngine = ComparisonEngine(dbEngine)

In [5]:
comparisonEngine.load('gender', sqlDateColumns=[], sparqlDateColumns=[])

SQL Columns:  [u'id' u'gender']
RDF Columns:  ['id' 'gender' 'genderValue']
SQL Count:  68
RDF Count:  68


In [6]:
comparisonEngine.interact(figsize=(18, 8))

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUneFNRTCcsIG9wdGlvbnM9KHUnaWQnLCB1J2dlbmRlcicpLCB2YWx1ZT11J2lkJyksIERyb3Bkb3duKGRlc2NyaXDigKY=
